-- Create Database
-- ================================
CREATE DATABASE IF NOT EXISTS carpool_app;
USE carpool_app;
 
-- ================================
-- USERS TABLE
-- ================================
CREATE TABLE users (
    user_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(150) UNIQUE NOT NULL,
    password VARCHAR(255) NOT NULL,
    role ENUM('driver','passenger','both') NOT NULL,
    is_active TINYINT(1) DEFAULT 1,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
);
 
-- ================================
-- DRIVERS TABLE
-- ================================
CREATE TABLE drivers (
    driver_id INT AUTO_INCREMENT PRIMARY KEY,
    user_id INT NOT NULL,
    avg_rating FLOAT DEFAULT 0,
    total_rides INT DEFAULT 0,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE
);
 
-- ================================
-- PASSENGERS TABLE
-- ================================
CREATE TABLE passengers (
    passenger_id INT AUTO_INCREMENT PRIMARY KEY,
    user_id INT NOT NULL,
    avg_rating FLOAT DEFAULT 0,
    total_rides INT DEFAULT 0,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE
);
 
-- ================================
-- ROUTES TABLE
-- ================================
CREATE TABLE routes (
    route_id INT AUTO_INCREMENT PRIMARY KEY,
    from_city VARCHAR(100) NOT NULL,
    to_city VARCHAR(100) NOT NULL,
    distance_km FLOAT NOT NULL,
    duration_min FLOAT NOT NULL,
    coordinates JSON NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
 
-- ================================
-- RIDE REQUESTS TABLE
-- ================================
CREATE TABLE ride_requests (
    request_id INT AUTO_INCREMENT PRIMARY KEY,
    passenger_id INT NOT NULL,
    from_city VARCHAR(100) NOT NULL,
    to_city VARCHAR(100) NOT NULL,
    date_time DATETIME NOT NULL,
    passengers_count INT NOT NULL,
    preferences JSON,
    status ENUM('pending','matched','active','completed','cancelled') DEFAULT 'pending',
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (passenger_id) REFERENCES passengers(passenger_id) ON DELETE CASCADE
);
 
-- ================================
-- RIDE OFFERS TABLE
-- ================================
CREATE TABLE ride_offers (
    offer_id INT AUTO_INCREMENT PRIMARY KEY,
    driver_id INT NOT NULL,
    vehicle_no VARCHAR(50) NOT NULL,
    route_id INT NOT NULL,
    request_id INT,
    accepted_at DATETIME,
    available_seats INT NOT NULL,
    price_per_km FLOAT NOT NULL,
    estimated_fare FLOAT,
    status ENUM('pending','open','matched','booked','active','completed','cancelled') DEFAULT 'pending',
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (driver_id) REFERENCES drivers(driver_id) ON DELETE CASCADE,
    FOREIGN KEY (route_id) REFERENCES routes(route_id) ON DELETE CASCADE,
    FOREIGN KEY (request_id) REFERENCES ride_requests(request_id) ON DELETE SET NULL
);
 
-- ================================
-- RIDES TABLE
-- ================================
CREATE TABLE rides (
    ride_id INT AUTO_INCREMENT PRIMARY KEY,
    offer_id INT NOT NULL,
    passenger_id INT NOT NULL,
    driver_id INT NOT NULL,
    seats_booked INT NOT NULL,
    total_fare FLOAT NOT NULL,
    start_time DATETIME,
    end_time DATETIME,
    status ENUM('pending','open','matched','booked','active','completed','cancelled') DEFAULT 'pending',
    current_position_index INT DEFAULT 0,
    FOREIGN KEY (offer_id) REFERENCES ride_offers(offer_id) ON DELETE CASCADE,
    FOREIGN KEY (passenger_id) REFERENCES passengers(passenger_id) ON DELETE CASCADE,
    FOREIGN KEY (driver_id) REFERENCES drivers(driver_id) ON DELETE CASCADE
);
 
-- ================================
-- RATINGS TABLE
-- ================================
CREATE TABLE ratings (
    rating_id INT AUTO_INCREMENT PRIMARY KEY,
    ride_id INT NOT NULL,
    rated_by INT NOT NULL,
    rated_user INT NOT NULL,
    rating FLOAT NOT NULL,
    feedback TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (ride_id) REFERENCES rides(ride_id) ON DELETE CASCADE,
    FOREIGN KEY (rated_by) REFERENCES users(user_id) ON DELETE CASCADE,
    FOREIGN KEY (rated_user) REFERENCES users(user_id) ON DELETE CASCADE
);
 
-- ================================
-- NOTIFICATIONS TABLE
-- ================================
CREATE TABLE notifications (
    notification_id INT AUTO_INCREMENT PRIMARY KEY,
    user_id INT NOT NULL,
    message TEXT NOT NULL,
    is_read TINYINT(1) DEFAULT 0,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE
);

-- ============================================
-- NEW: ride_incidents (Emergency events)
-- ============================================
CREATE TABLE ride_incidents (
    incident_id INT AUTO_INCREMENT PRIMARY KEY,
    ride_id INT NOT NULL,
    reported_by INT NOT NULL,
    incident_type ENUM(
        'emergency',
        'panic_stop',
        'suspicious',
        'danger',
        'mechanical_issue'
    ) NOT NULL,
    description TEXT,
    severity ENUM('low','medium','high','critical') DEFAULT 'low',
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
 
    FOREIGN KEY (ride_id) REFERENCES rides(ride_id) ON DELETE CASCADE,
    FOREIGN KEY (reported_by) REFERENCES users(user_id) ON DELETE CASCADE
);
 
-- ============================================
-- NEW: user_reports (Manual complaints)
-- ============================================
CREATE TABLE user_reports (
    report_id INT AUTO_INCREMENT PRIMARY KEY,
    reported_by INT NOT NULL,
    reported_user INT NULL,
    ride_id INT NULL,
    category ENUM(
        'driver_misconduct',
        'passenger_misconduct',
        'payment_issue',
        'safety_concern',
        'app_issue',
        'other'
    ) NOT NULL,
    description TEXT NOT NULL,
    status ENUM('open', 'in_review', 'resolved', 'dismissed') DEFAULT 'open',
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
 
    FOREIGN KEY (reported_by) REFERENCES users(user_id) ON DELETE CASCADE,
    FOREIGN KEY (reported_user) REFERENCES users(user_id) ON DELETE SET NULL,
    FOREIGN KEY (ride_id) REFERENCES rides(ride_id) ON DELETE SET NULL
);